In [1]:
import time
import datetime
import os
import numpy as np
from scipy.optimize import curve_fit
from scipy import stats
import gc
from mu2etools import *
from mu2etools import wideband
import math

hep.style.use('ATLAS')
DATASET="rec.mu2e.CRV_wideband_cosmics.CRVWB-000-008-000.root"

In [2]:
def aging(x, A, r):
    return A*(1-r/100)**x

In [3]:
DATASET="rec.mu2e.CRV_wideband_cosmics.CRVWB-000-008-000.root"

runs = [66, 94, 105, 119, 1010, 1020, 1021, 1022, 1031, 1033, 1034, 1035,
        1243, 1244, 1245, 1246, 1247, 1248, 1251, 1262, 1264, 1276, 1279, 1280, 1281, 1284, 1285, 1286, 1287, 1288,
        1312, 1315, 1327, 1341, 1354, 1370, 2044]
processor = wideband.DataProcessor(usexroot=True, treename='runSummary', runlist=runs, userunlist=True, remove=False,fixtimes=True)
filelist = processor.getFilelist(DATASET)
filelist = ["{}{}".format(i, ":runSummary") for i in filelist]

In [ ]:
varlist = ['runNumber', 'subrunNumber', 'timestamp', 'PEsTemperatureCorrected',
           'calibConstantsTemperatureCorrected', 'FWHMsTemperatureCorrected', 'signalsTemperatureCorrected',
          'meanTemperatures', 'stddevTemperatures']
d = {}
for chan in range(128):
    d["SPEs_{0}".format(chan)] = []
    d["PEs_{0}".format(chan)] = []
    d["PEs_sigma_{0}".format(chan)] = []
    d["temps_{0}".format(chan)] = []
    d["temps_sigma_{0}".format(chan)] = []
    d["time_{0}".format(chan)] = []
badchannels = []
    
for idx, filename in enumerate(filelist):
    with uproot.open(filename) as file:
        for ar in file.iterate(step_size="10MB", 
                               filter_name=varlist, 
                               report=False, library='ak'):
            ar['SPEsTemperatureCorrected'] = ak.mask(ar['calibConstantsTemperatureCorrected'], ar['calibConstantsTemperatureCorrected']>0)
            for chan in range(64):
                if ar['timestamp'][0] > 0:
                    if ar['PEsTemperatureCorrected'][0,0,chan] > 0:
                        d["SPEs_{0}".format(chan+64)].append(ar['SPEsTemperatureCorrected'][0,0,chan])
                        d["PEs_{0}".format(chan+64)].append(ar['PEsTemperatureCorrected'][0,0,chan])
                        d["PEs_sigma_{0}".format(chan+64)].append(ar['FWHMsTemperatureCorrected'][0,0,chan]/(2*math.sqrt(2*np.log(2)*ar['signalsTemperatureCorrected'][0,0,chan])))
                        d["temps_{0}".format(chan+64)].append(ar['meanTemperatures'][0,0,chan])
                        d["temps_sigma_{0}".format(chan+64)].append(ar['stddevTemperatures'][0,0,chan])
                        d["time_{0}".format(chan+64)].append(ar['timestamp'][0])
                    else:
                        badchannels.append(chan+64)
                    if ar['PEsTemperatureCorrected'][0,1,chan] > 0:
                        d["SPEs_{0}".format(chan)].append(ar['SPEsTemperatureCorrected'][0,1,chan])
                        d["PEs_{0}".format(chan)].append(ar['PEsTemperatureCorrected'][0,1,chan])
                        d["PEs_sigma_{0}".format(chan)].append(ar['FWHMsTemperatureCorrected'][0,1,chan]/(2*math.sqrt(2*np.log(2)*ar['signalsTemperatureCorrected'][0,1,chan])))
                        d["temps_{0}".format(chan)].append(ar['meanTemperatures'][0,1,chan])
                        d["temps_sigma_{0}".format(chan)].append(ar['stddevTemperatures'][0,1,chan])
                        d["time_{0}".format(chan)].append(ar['timestamp'][0])
                    else:
                        badchannels.append(chan)

for chan in range(128):
    if chan in badchannels:
        continue
    else:
        d["time_{0}".format(chan)] = np.array(d["time_{0}".format(chan)])
        d["dt_{0}".format(chan)] = (d["time_{0}".format(chan)] - d["time_{0}".format(chan)][0])/(60*60*24*365.25)
        d["SPEs_{0}".format(chan)] = np.array(d["SPEs_{0}".format(chan)])
        d["PEs_{0}".format(chan)] = np.array(d["PEs_{0}".format(chan)])
        d["temps_{0}".format(chan)] = np.array(d["temps_{0}".format(chan)])
        d["temps_sigma_{0}".format(chan)] = np.array(d["temps_sigma_{0}".format(chan)])
        d["PEs_sigma_{0}".format(chan)] = np.array(d["PEs_sigma_{0}".format(chan)])

In [ ]:
PE_aging = []
lowaging = []
highaging = []

for chan in range(128):
    if chan in badchannels:
        continue
        
    binned_stats = stats.binned_statistic(d["dt_{0}".format(chan)], d["PEs_{0}".format(chan)],
                                          statistic='mean', bins=18, range=(0,3))
    d["dt_binned_{0}".format(chan)] = (binned_stats[1][:-1] + binned_stats[1][1:])/2
    d["PEs_binned_{0}".format(chan)] = binned_stats[0]
    binned_stats = stats.binned_statistic(d["dt_{0}".format(chan)], d["PEs_sigma_{0}".format(chan)],
                                          statistic='mean', bins=18, range=(0,3))
    d["PEs_sigma_binned_{0}".format(chan)] = binned_stats[0]
    
    valid = ~(np.isnan(d["PEs_binned_{0}".format(chan)]) | np.isnan(d["dt_binned_{0}".format(chan)]) | np.isnan(d["PEs_sigma_binned_{0}".format(chan)]))
    
    popt, pcov = curve_fit(aging, d["dt_binned_{0}".format(chan)][valid], d["PEs_binned_{0}".format(chan)][valid],
                           sigma=d["PEs_sigma_binned_{0}".format(chan)][valid])
    PE_aging.append(popt[1])
    d["aging_{0}".format(chan)] = popt[1]

    if popt[1] < 1:
        print("low aging", chan)
        lowaging.append(chan)
    if popt[1] > 5:
        print("high aging", chan)
        highaging.append(chan)

    if chan%4 == 0:
        fig, ax0 = plt.subplots()
        ax0.errorbar(d['dt_binned_{0}'.format(chan)], d["PEs_binned_{0}".format(chan)], xerr=0.083, yerr=d["PEs_sigma_binned_{0}".format(chan)], fmt='.',markersize=1, color='k')
        ax0.plot(np.linspace(-0.1, 3.1, 100), aging(np.linspace(-0.1, 3.1, 100), popt[0], popt[1]), color='r')

        if chan < 64:
            FEB = 0
            channel = chan
        else:
            FEB = 1
            channel = chan - 64

        ax0.set_xlabel('Time [year]')

        ax0.set_ylabel('Light yield [PE]')
        ax0.set_title('Module 127 FEB{0} Ch{1} light yield'.format(int(FEB), int(channel)))

        ax0.set_xlim((-0.05, 3.05))

        ax0.set_ylim((min(d["PEs_{0}".format(chan)]) - 1, max(d["PEs_{0}".format(chan)]) + 1))

        ax0.text(.95, .95, 'Aging: {0}%'.format(round(popt[1], 2)), fontsize=18, ha='right', va='top', transform=ax0.transAxes)
        
        fig.subplots_adjust(hspace=0)
        fig.set_figheight(5)
        fig.set_figwidth(8)
        fig.show()

fig, ax0 = plt.subplots()
ax0.hist(PE_aging, bins = np.linspace(-1, 10, 23), histtype='step', linewidth=2)

ax0.set_title('Module 127')
ax0.set_xlabel('aging rate [%/yr]')

ax0.set_xlim((0,10))

fig.subplots_adjust(hspace=0)
fig.set_figheight(5)
fig.set_figwidth(8)
fig.show()

In [ ]:
for chan in range(128):
    if chan in badchannels:
        continue
        
    binned_stats = stats.binned_statistic(d["dt_{0}".format(chan)], d["SPEs_{0}".format(chan)],
                                          statistic='mean', bins=18, range=(0,3))
    d["SPEs_binned_{0}".format(chan)] = binned_stats[0]
    d["dt_binned_{0}".format(chan)] = (binned_stats[1][:-1] + binned_stats[1][1:])/2

    binned_stats = stats.binned_statistic(d["dt_{0}".format(chan)], d["PEs_{0}".format(chan)],
                                          statistic='mean', bins=18, range=(0,3))
    d["PEs_binned_{0}".format(chan)] = binned_stats[0]
    binned_stats = stats.binned_statistic(d["dt_{0}".format(chan)], d["PEs_sigma_{0}".format(chan)],
                                          statistic='mean', bins=18, range=(0,3))
    d["PEs_sigma_binned_{0}".format(chan)] = binned_stats[0]
    
    valid = ~(np.isnan(d["PEs_binned_{0}".format(chan)]) | np.isnan(d["dt_binned_{0}".format(chan)]) | np.isnan(d["PEs_sigma_binned_{0}".format(chan)]))
    
    popt, pcov = curve_fit(aging, d["dt_binned_{0}".format(chan)][valid], d["PEs_binned_{0}".format(chan)][valid],
                           sigma=d["PEs_sigma_binned_{0}".format(chan)][valid])
    PE_aging.append(popt[1])
    if chan%4 == 0:
        fig, ax0 = plt.subplots()
        ax1 = ax0.twinx()
        ax2 = ax0.twinx()
        ax1.plot(d['dt_binned_{0}'.format(chan)], d["SPEs_binned_{0}".format(chan)],'.', color='blue')
        ax2.plot(d['dt_{0}'.format(chan)], d["temps_{0}".format(chan)],color='green')
        ax0.errorbar(d['dt_binned_{0}'.format(chan)], d["PEs_binned_{0}".format(chan)], xerr=0.083, yerr=d["PEs_sigma_binned_{0}".format(chan)], fmt='.',markersize=1, color='k')
        ax0.plot(np.linspace(-0.1, 3.1, 100), aging(np.linspace(-0.1, 3.1, 100), popt[0], popt[1]), color='r')

        if chan < 64:
            FEB = 0
            channel = chan
        else:
            FEB = 1
            channel = chan - 64

        ax0.set_xlabel('Time [year]')

        ax1.set_ylabel('SPEs', color='blue')
        ax0.set_ylabel('Light yield [PE]', color='k')
        ax2.set_ylabel('Temperature [degC]',color='green')
        ax0.set_title('Module 127 FEB{0} Ch{1}'.format(int(FEB), int(channel)))

        ax0.set_xlim((-0.05, 3.05))
        ax1.set_xlim((-0.05, 3.05))

        ax1.set_ylim((min(d["SPEs_{0}".format(chan)]) - 1, max(d["SPEs_{0}".format(chan)]) + 1))
        ax0.set_ylim((min(d["PEs_{0}".format(chan)]) - 1, max(d["PEs_{0}".format(chan)]) + 1))

        ax0.text(.05, .95, 'Aging: {0}%'.format(round(popt[1], 2)), fontsize=18, ha='left', va='top', transform=ax0.transAxes)
        
        ax1.tick_params(axis='y', which='both', colors='b')
        ax2.tick_params(axis='y', which='both', colors='g')
        ax2.spines['right'].set_position(("axes", 1.18))
        
        fig.subplots_adjust(hspace=0)
        fig.set_figheight(5)
        fig.set_figwidth(10)
        fig.show()

In [ ]:
for chan in range(128):
    if chan in lowaging:
        binned_stats = stats.binned_statistic(d["dt_{0}".format(chan)], d["SPEs_{0}".format(chan)],
                                          statistic='mean', bins=18, range=(0,3))
        d["SPEs_binned_{0}".format(chan)] = binned_stats[0]
        d["dt_binned_{0}".format(chan)] = (binned_stats[1][:-1] + binned_stats[1][1:])/2

        binned_stats = stats.binned_statistic(d["dt_{0}".format(chan)], d["PEs_{0}".format(chan)],
                                              statistic='mean', bins=18, range=(0,3))
        d["PEs_binned_{0}".format(chan)] = binned_stats[0]
        binned_stats = stats.binned_statistic(d["dt_{0}".format(chan)], d["PEs_sigma_{0}".format(chan)],
                                              statistic='mean', bins=18, range=(0,3))
        d["PEs_sigma_binned_{0}".format(chan)] = binned_stats[0]

        valid = ~(np.isnan(d["PEs_binned_{0}".format(chan)]) | np.isnan(d["dt_binned_{0}".format(chan)]) | np.isnan(d["PEs_sigma_binned_{0}".format(chan)]))

        popt, pcov = curve_fit(aging, d["dt_binned_{0}".format(chan)][valid], d["PEs_binned_{0}".format(chan)][valid],
                               sigma=d["PEs_sigma_binned_{0}".format(chan)][valid])
        PE_aging.append(popt[1])
        
        fig, ax0 = plt.subplots()
        ax1 = ax0.twinx()
        ax2 = ax0.twinx()
        ax1.plot(d['dt_binned_{0}'.format(chan)], d["SPEs_binned_{0}".format(chan)],'.', color='blue')
        ax2.plot(d['dt_{0}'.format(chan)], d["temps_{0}".format(chan)],color='green')
        ax0.errorbar(d['dt_binned_{0}'.format(chan)], d["PEs_binned_{0}".format(chan)], xerr=0.083, yerr=d["PEs_sigma_binned_{0}".format(chan)], fmt='.',markersize=1, color='k')
        ax0.plot(np.linspace(-0.1, 3.1, 100), aging(np.linspace(-0.1, 3.1, 100), popt[0], popt[1]), color='r')

        if chan < 64:
            FEB = 0
            channel = chan
        else:
            FEB = 1
            channel = chan - 64

        ax0.set_xlabel('Time [year]')

        ax1.set_ylabel('SPEs', color='blue')
        ax0.set_ylabel('Light yield [PE]', color='k')
        ax2.set_ylabel('Temperature [degC]',color='green')
        ax0.set_title('Module 127 FEB{0} Ch{1}'.format(int(FEB), int(channel)))

        ax0.set_xlim((-0.05, 3.05))
        ax1.set_xlim((-0.05, 3.05))

        ax1.set_ylim((min(d["SPEs_{0}".format(chan)]) - 1, max(d["SPEs_{0}".format(chan)]) + 1))
        ax0.set_ylim((min(d["PEs_{0}".format(chan)]) - 1, max(d["PEs_{0}".format(chan)]) + 1))

        ax0.text(.05, .95, 'Aging: {0}%'.format(round(popt[1], 2)), fontsize=18, ha='left', va='top', transform=ax0.transAxes)
        
        ax1.tick_params(axis='y', which='both', colors='b')
        ax2.tick_params(axis='y', which='both', colors='g')
        ax2.spines['right'].set_position(("axes", 1.18))
        
        fig.subplots_adjust(hspace=0)
        fig.set_figheight(5)
        fig.set_figwidth(10)
        fig.show()

In [ ]:
for chan in range(128):
    if chan in highaging:
        binned_stats = stats.binned_statistic(d["dt_{0}".format(chan)], d["SPEs_{0}".format(chan)],
                                          statistic='mean', bins=18, range=(0,3))
        d["SPEs_binned_{0}".format(chan)] = binned_stats[0]
        d["dt_binned_{0}".format(chan)] = (binned_stats[1][:-1] + binned_stats[1][1:])/2

        binned_stats = stats.binned_statistic(d["dt_{0}".format(chan)], d["PEs_{0}".format(chan)],
                                              statistic='mean', bins=18, range=(0,3))
        d["PEs_binned_{0}".format(chan)] = binned_stats[0]
        binned_stats = stats.binned_statistic(d["dt_{0}".format(chan)], d["PEs_sigma_{0}".format(chan)],
                                              statistic='mean', bins=18, range=(0,3))
        d["PEs_sigma_binned_{0}".format(chan)] = binned_stats[0]

        valid = ~(np.isnan(d["PEs_binned_{0}".format(chan)]) | np.isnan(d["dt_binned_{0}".format(chan)]) | np.isnan(d["PEs_sigma_binned_{0}".format(chan)]))

        popt, pcov = curve_fit(aging, d["dt_binned_{0}".format(chan)][valid], d["PEs_binned_{0}".format(chan)][valid],
                               sigma=d["PEs_sigma_binned_{0}".format(chan)][valid])
        PE_aging.append(popt[1])
        
        fig, ax0 = plt.subplots()
        ax1 = ax0.twinx()
        ax2 = ax0.twinx()
        ax1.plot(d['dt_binned_{0}'.format(chan)], d["SPEs_binned_{0}".format(chan)],'.', color='blue')
        ax2.plot(d['dt_{0}'.format(chan)], d["temps_{0}".format(chan)],color='green')
        ax0.errorbar(d['dt_binned_{0}'.format(chan)], d["PEs_binned_{0}".format(chan)], xerr=0.083, yerr=d["PEs_sigma_binned_{0}".format(chan)], fmt='.',markersize=1, color='k')
        ax0.plot(np.linspace(-0.1, 3.1, 100), aging(np.linspace(-0.1, 3.1, 100), popt[0], popt[1]), color='r')

        if chan < 64:
            FEB = 0
            channel = chan
        else:
            FEB = 1
            channel = chan - 64

        ax0.set_xlabel('Time [year]')

        ax1.set_ylabel('SPEs', color='blue')
        ax0.set_ylabel('Light yield [PE]', color='k')
        ax2.set_ylabel('Temperature [degC]',color='green')
        ax0.set_title('Module 127 FEB{0} Ch{1}'.format(int(FEB), int(channel)))

        ax0.set_xlim((-0.05, 3.05))
        ax1.set_xlim((-0.05, 3.05))

        ax1.set_ylim((min(d["SPEs_{0}".format(chan)]) - 1, max(d["SPEs_{0}".format(chan)]) + 1))
        ax0.set_ylim((min(d["PEs_{0}".format(chan)]) - 1, max(d["PEs_{0}".format(chan)]) + 1))

        ax0.text(.05, .95, 'Aging: {0}%'.format(round(popt[1], 2)), fontsize=18, ha='left', va='top', transform=ax0.transAxes)
        
        ax1.tick_params(axis='y', which='both', colors='b')
        ax2.tick_params(axis='y', which='both', colors='g')
        ax2.spines['right'].set_position(("axes", 1.18))
        
        fig.subplots_adjust(hspace=0)
        fig.set_figheight(5)
        fig.set_figwidth(10)
        fig.show()